In [ ]:
# import bibliotek
import os
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Pobieramy dane z wybranej rywalizacji
container_check = os.getenv("iscontainer")
if container_check=="y":
    config_dir = Path("/home/vscode/.config/kaggle")
    config_dir.mkdir(parents=True, exist_ok=True)
    with open(config_dir / "kaggle.json", "w") as dst:
        with open("./kaggle.json", "r") as src:
            dst.write(src.read())


In [ ]:
# test czy Kaggle działa
api = KaggleApi()
api.authenticate()

# Poprawny sposób: bez limitu w funkcji
competitions = api.competitions_list()

# Wyświetlamy tylko 5 pierwszych ręcznie
for competition in competitions[:5]:
    print(competition.ref)


In [ ]:
# Autoryzacja
api = KaggleApi()
api.authenticate()

# Dataset
dataset = "kamilpytlak/personal-key-indicators-of-heart-disease"
download_folder = Path("../data")  # <-- UWAGA: idziemy 1 katalog wyżej (z notebooka)

# Upewnij się, że katalog 'data' istnieje
download_folder.mkdir(parents=True, exist_ok=True)

# Plik do pobrania
file_name = "2020/heart_2020_cleaned.csv"

# Pobranie tylko jednego pliku
api.dataset_download_file(dataset, file_name, path=str(download_folder))

# Ścieżka do pobranego pliku zip
zip_path = download_folder / "heart_2020_cleaned.csv.zip"

# Rozpakowanie pliku zip
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(str(download_folder))

# Usunięcie pliku zip po rozpakowaniu
os.remove(zip_path)


In [ ]:
!dir "../data"


In [ ]:
#  Wczytanie danych
path = "../data/heart_2020_cleaned.csv"
df = pd.read_csv(path)

In [ ]:
# Sprawdzenie danych w zakresie ilości "zawałów" serca
df["HeartDisease"].value_counts()

In [ ]:
# Wyświetlwnie pierwszych 5 wierszy
print("Pierwsze 5 wierszy danych:")
print(df.head())

In [ ]:
# Informacje o zbiorze danych
print("Informacje o zbiorze danych:")
print(df.info())

In [ ]:
# Statystyki opisowe

print("\n\u2794 Statystyki opisowe:")
print(df.describe())

# znacznik strzałki "\n\u2794 "

In [ ]:
# Sprawdzenie brakujących danych
print("Sprawdzenie brakujących danych:")
print(df.isnull().sum())

In [ ]:
# Rozkład zmiennych
# Wykres rozkładu dla zmiennej docelowej (target)
os.makedirs("../eksploracja", exist_ok=True)
sns.countplot(data=df, x="HeartDisease")
plt.title("Rozkład zmiennej docelowej - Heart Disease")
#plt.savefig("../eksploracja/Rozklad_zmiennej_docelowej.png", dpi=300, bbox_inches='tight')
plt.savefig("../eksploracja/Rozklad_zmiennej_docelowej.png")
plt.show()


In [ ]:
# Macierz korelacji dla zmiennych numerycznych
os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Macierz korelacji zmiennych numerycznych")
plt.savefig("../eksploracja/Macierz_korelacji.png")
plt.show()

In [ ]:
# Lista unikalnych wartości dla zmiennych kategorycznych
print("Unikalne wartości w zmiennych kategorycznych:")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Wykresy słupkowe dla zmiennych kategorycznych

os.makedirs("../eksploracja", exist_ok=True)
categorical_cols = df.select_dtypes(include='object').columns

for col in categorical_cols:
    plt.figure(figsize=(8, 4))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index)
    plt.title(f"Rozkład zmiennej: {col}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"../eksploracja/{col}.png")
    plt.show()

In [ ]:
# Tabele kontyngencji względem HeartDisease
for col in categorical_cols:
    print(f"\nTabela krzyżowa: {col} vs HeartDisease")
    print(pd.crosstab(df[col], df["HeartDisease"], normalize='index') * 100)

In [ ]:
# Wczytanie danych
#df = pd.read_csv("ścieżka/do/heart_2020_cleaned.csv")  # <-- podmień na własną ścieżkę, np. "../data/heart_2020_cleaned.csv"

# Statystyki opisowe dla danych liczbowych
print("Statystyki opisowe danych liczbowych:")
print(df.describe())

# Statystyki opisowe dla danych kategorycznych
print("\nStatystyki opisowe danych kategorycznych:")
print(df.describe(include=["object"]))


In [ ]:
# Wizualizacja: Rozkład wartości BMI
os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(10, 6))
sns.histplot(df['BMI'], kde=True)
plt.title('Rozkład wartości BMI')
plt.xlabel('BMI')
plt.ylabel('Liczba osób')
plt.grid(True)
plt.savefig("../Rozkład_wartości_BMI.png")
plt.show()

In [ ]:
# Wizualizacja: Choroba serca względem kategorii wiekowej

os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(14, 6))
sns.countplot(data=df, x='AgeCategory', hue='HeartDisease')
plt.title('Występowanie choroby serca w zależności od wieku')
plt.xlabel('Kategoria wiekowa')
plt.ylabel('Liczba przypadków')
plt.xticks(rotation=45)
plt.grid(True)
plt.savefig("../eksploracja/Występowanie_choroby_serca_w_zależności_od_wieku.png")  
plt.show()

In [ ]:
# Spanie
os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(10, 6))
sns.histplot(df['SleepTime'], kde=True)
plt.title('Rozkład godzin snu')
plt.xlabel('Liczba godzin snu')
plt.ylabel('Liczba osób')
plt.grid(True)
plt.savefig("../eksploracja/Rozkład_godzin_snu.png")    
plt.show()

In [ ]:
# Cukrzyca
os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Diabetic')
plt.title('Występowanie cukrzycy')
plt.xlabel('Czy osoba choruje na cukrzycę?')
plt.ylabel('Liczba osób')
plt.grid(True)
plt.savefig("../eksploracja/Występowanie_cukrzycy.png")
plt.show()

In [ ]:
# Wizualizacja: Zdrowie ogólne vs choroby serca
os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='GenHealth', hue='HeartDisease')
plt.title('GenHealth a występowanie choroby serca')
plt.xlabel('Ocena ogólnego zdrowia')
plt.ylabel('Liczba osób')
plt.grid(True)
plt.savefig("../eksploracja/GenHealth_a_występowanie_choroby_serca.png")
plt.show()

In [ ]:
# Wizualizacja: Palenie względem chorób serca
os.makedirs("../eksploracja", exist_ok=True)
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Smoking', hue='HeartDisease')
plt.title('Palenie tytoniu a występowanie choroby serca')
plt.xlabel('Czy osoba pali papierosy?')
plt.ylabel('Liczba osób')
plt.grid(True)
plt.savefig("../eksploracja/Palenie_tytoniu_a_występowanie_choroby_serca.png")
plt.show()

In [ ]:
# Podsumowanie
print("Eksploracja danych zakończona.")